In [42]:
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader,JSONLoader
from langchain_ollama import OllamaEmbeddings,OllamaLLM
import json
import requests



In [43]:
model = OllamaLLM(model="gemma2:2b")

from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")




In [44]:
len(chunks)

5386

In [45]:
import os
import requests
from bs4 import BeautifulSoup
from langchain.schema import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Optional: PyPDFLoader fallback
from langchain_community.document_loaders import PyPDFLoader
import pdfplumber
from pypdf import PdfReader

# -----------------------------
# JSON manifest
# -----------------------------
manifest = [
    {
        "title": "NX Nastran Basic Dynamic Analysis User’s Guide",
        "url": "https://www.smart-fem.de/media/basic_dynamics.pdf",
        "vendor": "Siemens (NX Nastran)"
    },
    {
        "title": "NX Nastran Error List",
        "url": "https://www.scribd.com/document/447941028/NX-Nastran-Error-List",
        "vendor": "Siemens (NX Nastran)"
    },
    {
        "title": "NX Nastran Users Guide",
        "url": "https://www.scribd.com/document/155616253/NX-Nastran-Users-Guide",
        "vendor": "Siemens (NX Nastran)"
    },
    {
        "title": "NX Nastran Element Library Reference Manual",
        "url": "https://www.scribd.com/document/357044977/NX-Nastran-Element-Library-Reference-Manual",
        "vendor": "Siemens (NX Nastran)"
    },
    {
        "title": "NX Nastran Basic Fact Sheet",
        "url": "https://www.plm.automation.siemens.com/en_gb/Images/nx%20nastran%20basic%20fs%20W%203_tcm642-4481.pdf",
        "vendor": "Siemens (NX Nastran)"
    },
    {
        "title": "Structural Analysis Using NX Nastran 9.0",
        "url": "https://ntrs.nasa.gov/api/citations/20140016797/downloads/20140016797.pdf",
        "vendor": "NASA"
    },
    {
        "title": "MSC Nastran 2024.1 Quick Reference Guide",
        "url": "https://nexus.hexagon.com/documentationcenter/en-US/bundle/MSC_Nastran_2024.1_Quick_Reference_Guide/resource/MSC_Nastran_2024.1_Quick_Reference_Guide.pdf",
        "vendor": "MSC Software"
    },
    {
        "title": "NX Nastran – Direct Matrix Abstraction Program (DMAP)",
        "url": "https://www.ata-e.com/wp-content/uploads/2024/10/NXNastran_DMAP.pdf",
        "vendor": "Siemens (NX Nastran)"
    },
    {
        "title": "NX Nastran Superelements",
        "url": "https://www.ata-e.com/wp-content/uploads/2024/10/Siemens-PLM-NX-Nastran-Superelements-fs-2652-A10.pdf",
        "vendor": "Siemens (NX Nastran)"
    },
    {
        "title": "NX Nastran 8 Release Guide",
        "url": "https://www.smart-fem.de/media/aktuelles/nxnast8releaseguide.pdf",
        "vendor": "Siemens (NX Nastran)"
    }
]

pdf_folder = r"F:\VS code\NX-DEBBUGGER-RAG_APP\pdf_folder"
os.makedirs(pdf_folder, exist_ok=True)
docs = []

# -----------------------------
# Function to load PDF
# -----------------------------
def load_pdf_with_fallback(pdf_path):
    # Try PyPDFLoader first
    try:
        loader = PyPDFLoader(pdf_path)
        return loader.load()
    except Exception:
        pass

    # Try pypdf
    try:
        reader = PdfReader(pdf_path)
        pages = []
        for page in reader.pages:
            pages.append(Document(page_content=page.extract_text()))
        return pages
    except Exception:
        pass

    # Try pdfplumber
    try:
        pages = []
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                pages.append(Document(page_content=page.extract_text()))
        return pages
    except Exception:
        pass

    # If all fail
    return []

# -----------------------------
# Process manifest
# -----------------------------
for i, entry in enumerate(manifest, start=1):
    url = entry.get("url")
    title = entry.get("title")
    vendor = entry.get("vendor")

    if url.lower().endswith(".pdf"):
        file_name = f"nastran_doc_{i}.pdf"
        pdf_path = os.path.join(pdf_folder, file_name)

        # Download PDF
        if not os.path.exists(pdf_path):
            try:
                r = requests.get(url, stream=True, timeout=15)
                r.raise_for_status()
                with open(pdf_path, "wb") as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        f.write(chunk)
                print(f"Downloaded PDF: {file_name}")
            except Exception as e:
                print(f"Failed to download PDF {url}: {e}")
                continue

        # Load PDF using fallback method
        pdf_docs = load_pdf_with_fallback(pdf_path)
        if pdf_docs:
            for doc in pdf_docs:
                doc.metadata.update({
                    "title": title,
                    "vendor": vendor,
                    "source_url": url,
                    "filename": file_name
                })
            docs.extend(pdf_docs)
            print(f"Loaded {len(pdf_docs)} pages from {file_name}")
        else:
            print(f"Failed to parse PDF {file_name}, trying webpage fallback...")

            # Attempt HTML fetch if PDF fails
            try:
                r = requests.get(url, timeout=15)
                r.raise_for_status()
                soup = BeautifulSoup(r.text, "html.parser")
                text = soup.get_text(separator="\n", strip=True)
                docs.append(Document(
                    page_content=text,
                    metadata={
                        "title": title,
                        "vendor": vendor,
                        "source_url": url
                    }
                ))
                print(f"Loaded webpage fallback for: {title}")
            except Exception as e:
                print(f"Failed webpage fallback for {url}: {e}")

    else:  # Webpage
        try:
            r = requests.get(url, timeout=15)
            r.raise_for_status()
            soup = BeautifulSoup(r.text, "html.parser")
            text = soup.get_text(separator="\n", strip=True)
            docs.append(Document(
                page_content=text,
                metadata={
                    "title": title,
                    "vendor": vendor,
                    "source_url": url
                }
            ))
            print(f"Loaded webpage content: {title}")
        except Exception as e:
            print(f"Failed to fetch URL {url}: {e}")

# -----------------------------
# Split documents into chunks
# -----------------------------
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = splitter.split_documents(docs)
print(f"Created {len(chunks)} chunks for RAG indexing")


invalid pdf header: b'<!DOC'
EOF marker not found
invalid pdf header: b'<!DOC'
EOF marker not found


Failed to parse PDF nastran_doc_1.pdf, trying webpage fallback...
Loaded webpage fallback for: NX Nastran Basic Dynamic Analysis User’s Guide
Loaded webpage content: NX Nastran Error List
Loaded webpage content: NX Nastran Users Guide
Loaded webpage content: NX Nastran Element Library Reference Manual
Loaded 6 pages from nastran_doc_5.pdf


invalid pdf header: b'<!DOC'
EOF marker not found
invalid pdf header: b'<!DOC'
EOF marker not found


Loaded 9 pages from nastran_doc_6.pdf
Failed to parse PDF nastran_doc_7.pdf, trying webpage fallback...
Loaded webpage fallback for: MSC Nastran 2024.1 Quick Reference Guide
Loaded 2 pages from nastran_doc_8.pdf
Loaded 2 pages from nastran_doc_9.pdf
Loaded 345 pages from nastran_doc_10.pdf
Created 5386 chunks for RAG indexing


In [52]:
db=FAISS.from_documents(embedding=embedding_model,documents=chunks)
db.save_local(r"F:\\VS code\\NX-DEBBUGGER-RAG_APP")

In [64]:
db = FAISS.load_local(r"F:\\VS code\\NX-DEBBUGGER-RAG_APP\\faiss_index", embeddings=embedding_model,allow_dangerous_deserialization=True)
retriever=db.as_retriever(search_kwargs={"k":5})
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001D443436AA0>, search_kwargs={'k': 5})

In [65]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA


prompt_template = """
You are an expert NX Nastran debugging assistant.

Use the retrieved documentation to answer the user’s question. 
Always provide a clear explanation, step-by-step guidance, and possible solutions.

Question: {question}

Reference information from the documents:
{context}

Answer:
"""
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["question", "context"]
)


qa_chain = RetrievalQA.from_chain_type(
    llm=model,          # your Gemma 2 or other LLM instance
    retriever=retriever,     
    return_source_documents=True,
    chain_type="stuff",      # or "map_reduce" for longer contexts
    chain_type_kwargs={"prompt": prompt},
    input_key="question" 
    )


In [66]:
query_text = "How do I fix no go error"

output = qa_chain({"question": query_text})  # Use 'question' instead of 'query'

print("Answer:\n", output['result'])


Answer:
 It seems like you're asking about "no go error" in NX Nastran. Here's a breakdown of potential causes and solutions, along with explanations for understanding these errors:

**Understanding "No Go Error" in NX Nastran**

A "no go error" typically means your model or analysis setup failed to run successfully. This can be caused by various factors: 

* **Geometric Discretization Problems:**  NX Nastran uses complex algorithms and requires a precise mesh of the structure being analyzed. Errors here, like too many elements on a boundary, are common.
* **Dynamic Solver Challenges:** If you're simulating dynamic loads (e.g., impact analysis), it can lead to "no go" errors if your model has extremely large displacements or very stiff structures. 
* **Computational Issues:**  The computer may not have enough resources for the calculations involved in the model – especially with complex simulations.

**Troubleshooting Steps**

1. **Read the Exact Error Message:** The most important ste